# Chapter 07: 학습 (Learning)

이 노트북에서는 Reflexion 패턴을 통한 에이전트 자기 개선을 학습합니다.

## 주요 내용
- Reflexion: 실패로부터 학습
- 시행착오 기반 개선
- 경험적 학습

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/your-repo/notebook/ch07_learning.ipynb)


In [ ]:
!pip install -q langchain langchain-openai langgraph python-dotenv


In [ ]:
import os
try:
    from google.colab import userdata
    os.environ["OPENAI_API_KEY"] = userdata.get('OPENAI_API_KEY')
except:
    pass
if not os.getenv("OPENAI_API_KEY"):
    os.environ["OPENAI_API_KEY"] = "sk-your-api-key-here"


## Reflexion: 실패로부터 학습하기

Reflexion은 에이전트가 실패한 작업을 분석하고 다음 시도에서 개선할 수 있도록 하는 학습 패턴입니다.


In [ ]:
from langchain.chat_models import init_chat_model
from langgraph.graph import StateGraph, MessagesState, START
from langchain_core.messages import HumanMessage

# LLM 초기화
llm = init_chat_model(model="gpt-5-mini", temperature=0)

def call_model(state: MessagesState):
    response = llm.invoke(state["messages"])
    return {"messages": response}

# Reflexion 프롬프트 예시
reflexion_prompt = """어떤 환경에 놓인 뒤 특정 과제를 수행하라는 지시를 받았던 과거 경험의 기록이 주어질 것입니다.
당신은 그 과제를 완수하는 데 실패했습니다. 환경을 요약하려고 하지 말고, 과제를 완수하기 위해 시도했던 전략과 진행 경로에 대해 생각해 보십시오.

Instruction:
저는 유제품이 들어 있지 않고 사과 맛이 포함된 칩스 버라이어티 팩을 찾고 있으며, 가격은 30달러 미만이어야 합니다.

[이전 시도]
Action: search[dairy free and apple variety pack of chips]
Observation: 검색 결과 3개 발견
Action: click[B008D2X8C4] (가격: $100.0)
Action: click[Buy Now]
STATUS: FAIL (예산 초과)

이 실패를 분석하고, 다음 시도를 위한 계획을 세우세요.
Plan:"""

print("📝 Reflexion 프롬프트 준비 완료")


In [ ]:
# Reflexion 그래프 구성 및 실행
builder = StateGraph(MessagesState)
builder.add_node("reflexion", call_model)
builder.add_edge(START, "reflexion")
graph = builder.compile()

result = graph.invoke({"messages": [HumanMessage(reflexion_prompt)]})

print("=" * 60)
print("🔄 Reflexion 결과")
print("=" * 60)
for msg in result["messages"]:
    if msg.__class__.__name__ == "AIMessage":
        print("\n📋 AI 분석 및 개선 계획:")
        print("-" * 60)
        print(msg.content)
